In [1]:
from load_utils import prepare_data
from train_utils import *
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook
from Calculate_BLEU import *
import datetime

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [3]:
q, a, pairs, vector = prepare_data('test', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading test -------
Read 4041 sentence pairs
Counting words
Counted words:
In questions: 6821 words
In answers: 4758 words


In [4]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except:
        continue

In [5]:
from matplotlib import pyplot as plt

class LivePlotNotebook(object):
    """
    Live plot using %matplotlib notebook in jupyter notebook
    
    Usage:
    ```
    import time
    liveplot = LivePlotNotebook()
    x=np.random.random((10,))
    for i in range(10):
        time.sleep(1)
        liveplot.update(
            x=x+np.random.random(x.shape)/10,
            actions=np.random.randint(0, 3, size=(10,))
        )
    ```
    
    url:
    """

    def __init__(self):
        %matplotlib notebook
        fig,ax = plt.subplots(1,1)
        
        ax.plot([0]*20, label='price')
        ax.plot([1]*20, [1]*20, 'o', ms=12,c='gray', label='hold')
        
        
    
        ax.legend()
        ax.set_xlabel('timesteps')
        ax.grid()
        ax.set_title('actions')
        
        self.ax = ax
        self.fig = fig

    def update(self, x, actions):             
        # update price
        line = self.ax.lines[0]
        line.set_xdata(range(len(x)))
        line.set_ydata(x)
        
        # update action plots
        for i, line in enumerate(self.ax.lines[1:]):
            line.set_xdata(np.argwhere(actions==i).T)
            line.set_ydata(x[actions==i])
            line.set_marker(['o','^','v'][i])

        # update limits
        self.ax.set_xlim(0, len(actions))
        self.ax.set_ylim(x.min(), x.max())

        self.fig.canvas.draw()
        

In [6]:
criterion = nn.NLLLoss()
optimizer = 'Adam'
batch_size = 32

In [7]:
model = Seq2Seq(batch_size, q.n_words, a.n_words, 300, 100, weights_matrix, 0.5, device, criterion)

C:\Users\maren\Anaconda3\envs\deeplearning\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
a.n_words

4758

In [9]:
for epoch in tnrange(50, desc="Total epochs: "):

    print(f'Epoch {epoch + 1}: {datetime.datetime.now()}')
  
    # Calculte loss
    loss = epoch_train(model, optimizer, batch_size, pairs, q, a, device)

    print(f'Loss: {loss}')

    # Calculate accuracy
    accuracy = epoch_accuray(model, batch_size, pairs, q, a, device)

    print(f'Train accuracy: {accuracy}')

    # Calculate BLEU Score
    BLEU_model = CalculateBleu(model, batch_size, pairs, q, a, device)
    bleu_score = BLEU_model.score()

    print(f'BLUE score: {bleu_score}')

    # Try to do interactive plot
    


print(f"Optimization ended successfully")


Total epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1: 2021-09-03 18:26:42.114880
Loss: 5.43847137498195
Train accuracy: 0.0
BLUE score: 0
Epoch 2: 2021-09-03 18:27:01.743034
Loss: 2.340335253293696
Train accuracy: 0.0
BLUE score: 0
Epoch 3: 2021-09-03 18:27:20.421369
Loss: 2.1794790256018075
Train accuracy: 0.0
BLUE score: 0.0007732466684332356
Epoch 4: 2021-09-03 18:27:39.068306
Loss: 2.1338159802105015
Train accuracy: 0.0
BLUE score: 0.0016905299428081337
Epoch 5: 2021-09-03 18:27:57.886166
Loss: 2.061749776049418
Train accuracy: 0.0
BLUE score: 0.0008146006536487812
Epoch 6: 2021-09-03 18:28:16.768908
Loss: 2.0306759330062576
Train accuracy: 0.0
BLUE score: 0.0010536546538829656
Epoch 7: 2021-09-03 18:28:35.754457
Loss: 1.952253047206846
Train accuracy: 0.00024746349913387774
BLUE score: 0.0010198705192115476
Epoch 8: 2021-09-03 18:28:54.896279
Loss: 1.9790495603047016
Train accuracy: 0.0
BLUE score: 0.0013464341684969988
Epoch 9: 2021-09-03 18:29:14.104190
Loss: 1.9518401032486998
Train accuracy: 0.0
BLUE score: 0.00081775646